In [86]:
#modules for data preprocessing
import pandas as pd
import numpy as np
import re
import string


import nltk
#from nltk.corpus import stopwords
#from nltk.tokenize import word_tokenize 
#from nltk.stem.porter import *
#from nltk.stem.wordnet import WordNetLemmatizer

#modules for feature representation
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

#modules for training 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, cross_val_score, cross_val_predict

from sklearn.linear_model import LogisticRegression
#from sklearn.naive_bayes import MultinomialNB
#from sklearn.ensemble import RandomForestClassifier
#from sklearn.model_selection import GridSearchCV

from numpy import mean
from numpy import std
import sklearn.metrics as matrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


In [87]:
#load the training data
pd.set_option('display.max_colwidth', 100)

df1 = pd.read_csv('/Users/dufflelee/COS80027/assignment1/datasets/x_train.csv', header=None, dtype=str)
df2 = pd.read_csv('/Users/dufflelee/COS80027/assignment1/datasets/y_train.csv', header=None, dtype=str)

df1.columns = ['category', 'msg']
df2.columns = ['Sentiment']


print(df1, df2)
df1.head()

     category  \
0      amazon   
1      amazon   
2      amazon   
3      amazon   
4      amazon   
...       ...   
2395     yelp   
2396     yelp   
2397     yelp   
2398     yelp   
2399     yelp   

                                                                                                      msg  
0                            Oh and I forgot to also mention the weird color effect it has on your phone.  
1                                                                            THAT one didn't work either.  
2                                                                                      Waste of 13 bucks.  
3     Product is useless, since it does not have enough charging current to charge the 2 cellphones I ...  
4                               None of the three sizes they sent with the headset would stay in my ears.  
...                                                                                                   ...  
2395                                   

,category,msg
0,amazon,Oh and I forgot to also mention the weird color effect it has on your phone.
1,amazon,THAT one didn't work either.
2,amazon,Waste of 13 bucks.
3,amazon,"Product is useless, since it does not have enough charging current to charge the 2 cellphones I ..."
4,amazon,None of the three sizes they sent with the headset would stay in my ears.


In [88]:
#remove punctuation in x_train.csv

def remove_punctuation(msg):
    msg_nopunt = "".join([c for c in msg if c not in string.punctuation])
    return msg_nopunt

df1['msg'] = df1['msg'].apply(lambda x: remove_punctuation(x))

print (df1)

df1.head()

     category  \
0      amazon   
1      amazon   
2      amazon   
3      amazon   
4      amazon   
...       ...   
2395     yelp   
2396     yelp   
2397     yelp   
2398     yelp   
2399     yelp   

                                                                                                      msg  
0                             Oh and I forgot to also mention the weird color effect it has on your phone  
1                                                                              THAT one didnt work either  
2                                                                                       Waste of 13 bucks  
3     Product is useless since it does not have enough charging current to charge the 2 cellphones I w...  
4                                None of the three sizes they sent with the headset would stay in my ears  
...                                                                                                   ...  
2395                                   

,category,msg
0,amazon,Oh and I forgot to also mention the weird color effect it has on your phone
1,amazon,THAT one didnt work either
2,amazon,Waste of 13 bucks
3,amazon,Product is useless since it does not have enough charging current to charge the 2 cellphones I w...
4,amazon,None of the three sizes they sent with the headset would stay in my ears


In [89]:
#convert to lower case in x_train.csv
df1 = df1.apply(lambda x: x.astype(str).str.lower())

print (df1)

df1.head()


     category  \
0      amazon   
1      amazon   
2      amazon   
3      amazon   
4      amazon   
...       ...   
2395     yelp   
2396     yelp   
2397     yelp   
2398     yelp   
2399     yelp   

                                                                                                      msg  
0                             oh and i forgot to also mention the weird color effect it has on your phone  
1                                                                              that one didnt work either  
2                                                                                       waste of 13 bucks  
3     product is useless since it does not have enough charging current to charge the 2 cellphones i w...  
4                                none of the three sizes they sent with the headset would stay in my ears  
...                                                                                                   ...  
2395                                   

,category,msg
0,amazon,oh and i forgot to also mention the weird color effect it has on your phone
1,amazon,that one didnt work either
2,amazon,waste of 13 bucks
3,amazon,product is useless since it does not have enough charging current to charge the 2 cellphones i w...
4,amazon,none of the three sizes they sent with the headset would stay in my ears


In [90]:
#vectoriz and count matrix using CountVectorizer
count_vect = CountVectorizer(stop_words='english',min_df=2, max_df=0.75, ngram_range=(1, 2), binary=False)
count_matrix = count_vect.fit_transform(df1.msg)
count_array = count_matrix.toarray()

pd.DataFrame(data=count_array,columns=count_vect.get_feature_names())


,010,10,10 10,10 minutes,100,10oct,110,12,13,15,...,years ago,yes,youd,youd expect,youll,young,youre,yummy,zero,zero stars
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2395,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2396,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2397,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2398,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [91]:
print("Vocabulary size: {}".format(len(count_vect.vocabulary_)))
print("Vocabulary content:\n {}".format(count_vect.vocabulary_))


bag_of_words = count_vect.transform(df1.msg)
print(bag_of_words)


count_vect.get_feature_names()

Vocabulary size: 2081
Vocabulary content:
 {'oh': 1253, 'forgot': 735, 'mention': 1163, 'weird': 2004, 'color': 329, 'effect': 569, 'phone': 1318, 'didnt': 458, 'work': 2037, 'didnt work': 462, 'waste': 1982, '13': 8, 'bucks': 223, 'product': 1423, 'useless': 1935, 'does': 499, 'charging': 284, 'current': 399, 'charge': 280, 'use': 1931, 'charging current': 285, 'sizes': 1677, 'headset': 862, 'stay': 1742, 'ears': 554, 'worst': 2050, 'customer': 401, 'service': 1639, 'customer service': 402, 'lacking': 1024, 'makes': 1129, 'sound': 1703, 'says': 1589, 'signal': 1666, 'failed': 634, 'disappointing': 480, 'thing': 1830, 'disappointed': 479, 'basically': 143, 'bad': 126, 'choice': 306, 'disappoint': 478, 'red': 1513, 'horrible': 903, 'switch': 1794, 'times': 1859, 'feels': 670, 'poorly': 1377, 'constructed': 358, 'difficult': 466, 'buttons': 236, 'push': 1447, 'poorly constructed': 1378, 'dont': 508, 'make': 1126, 'mistake': 1179, 'did': 453, 'dont make': 515, 'make mistake': 1128, 'mista

['010',
 '10',
 '10 10',
 '10 minutes',
 '100',
 '10oct',
 '110',
 '12',
 '13',
 '15',
 '20',
 '25',
 '25 years',
 '30',
 '30 minutes',
 '35',
 '40',
 '40 minutes',
 '45',
 '50',
 '510',
 '5year',
 '80s',
 '90',
 '90 minutes',
 '90s',
 'ability',
 'able',
 'absolutely',
 'absolutely amazing',
 'absolutely loved',
 'access',
 'accused',
 'acted',
 'acting',
 'acting bad',
 'action',
 'action scenes',
 'actor',
 'actors',
 'actress',
 'actresses',
 'actually',
 'adaptation',
 'addition',
 'adorable',
 'advertised',
 'advise',
 'aerial',
 'aerial scenes',
 'africa',
 'afternoon',
 'age',
 'ago',
 'air',
 'allow',
 'allowing',
 'allows',
 'amazed',
 'amazing',
 'amazing film',
 'amazon',
 'ambiance',
 'ambience',
 'american',
 'amp',
 'amp got',
 'amp just',
 'ample',
 'angeles',
 'angle',
 'animation',
 'annoying',
 'anytime',
 'anytime soon',
 'apart',
 'appalling',
 'apparently',
 'appealing',
 'appearance',
 'appears',
 'appetizers',
 'appreciate',
 'appreciate subtle',
 'area',
 'aren

In [129]:
#normalize the matrix using TfidfTransformer

tf_transformer = TfidfTransformer(use_idf=True, smooth_idf=True).fit(count_matrix)
count_matrix_tf = tf_transformer.transform(count_matrix)
#cont_matrix_tf_array = count_matrix_tf.toarray()

print(count_matrix_tf)

pd.DataFrame(count_matrix_tf.toarray(),columns=count_vect.get_feature_names())

  (0, 2004)	0.4022968682104578
  (0, 1318)	0.2183181927862526
  (0, 1253)	0.36933644809291993
  (0, 1163)	0.37594118850260555
  (0, 735)	0.4309416609073697
  (0, 569)	0.4309416609073697
  (0, 329)	0.37594118850260555
  (1, 2037)	0.4879501338215343
  (1, 462)	0.7153303301036872
  (1, 458)	0.5002071428291452
  (2, 1982)	0.455079396158039
  (2, 223)	0.617521659376827
  (2, 8)	0.64153701638559
  (3, 1935)	0.36881327933575886
  (3, 1931)	0.2802744293488948
  (3, 1423)	0.27024450471394096
  (3, 499)	0.2998959314344918
  (3, 399)	0.42277093338641736
  (3, 285)	0.42277093338641736
  (3, 284)	0.3846393262182982
  (3, 280)	0.34210438715865465
  (4, 1742)	0.4909042927478763
  (4, 1677)	0.5820986078869763
  (4, 862)	0.3897853077066946
  (4, 554)	0.5179204571686168
  :	:
  (2394, 499)	0.35342807473189525
  (2395, 1793)	0.4596553785798142
  (2395, 1792)	0.408976968143334
  (2395, 1608)	0.4596553785798142
  (2395, 1389)	0.44244859590687546
  (2395, 789)	0.21713770958058562
  (2395, 745)	0.40897696814

,010,10,10 10,10 minutes,100,10oct,110,12,13,15,...,years ago,yes,youd,youd expect,youll,young,youre,yummy,zero,zero stars
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.641537,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2395,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2396,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2397,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2398,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [130]:
#data spliting 
x_train, x_test, y_train, y_test = train_test_split(count_matrix_tf.toarray(), df2['Sentiment'], test_size=0.1, random_state=1)

print (x_train, y_train)
print (x_test, y_test)


[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]] 1719    0
639     1
1595    1
612     1
2035    1
       ..
960     0
905     0
1096    0
235     0
1061    0
Name: Sentiment, Length: 2160, dtype: object
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]] 1348    1
468     1
1463    1
2267    1
943     0
       ..
1578    1
1610    0
614     1
1758    0
772     1
Name: Sentiment, Length: 240, dtype: object


In [132]:
#Train the xxx model on the training set

logreg = LogisticRegression(C=2, n_jobs=1, solver='lbfgs', class_weight='balanced')

cv = KFold(n_splits=3)

scores = cross_val_score(logreg, x_train, y_train, scoring='accuracy', cv=cv, n_jobs=1)
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

predictions = cross_val_predict(logreg, x_test, y_test, cv=cv)
print(predictions)
print(predictions.shape)



logreg.fit(x_train, y_train)
print('Training set score: {:.3f}'.format(logreg.score(x_train, y_train)))
print('Test set score: {:.3f}'.format(logreg.score(x_test, y_test)))

Accuracy: 0.798 (0.010)
['1' '1' '1' '0' '0' '0' '0' '0' '0' '1' '0' '0' '1' '0' '1' '0' '1' '1'
 '1' '0' '0' '0' '1' '1' '0' '0' '0' '0' '0' '1' '1' '1' '0' '0' '0' '0'
 '0' '1' '0' '0' '0' '0' '0' '0' '1' '1' '0' '0' '1' '0' '0' '1' '1' '0'
 '0' '0' '0' '0' '1' '0' '1' '0' '0' '0' '0' '1' '0' '1' '0' '1' '1' '1'
 '0' '1' '1' '0' '1' '1' '0' '1' '1' '1' '0' '0' '1' '1' '0' '1' '0' '0'
 '0' '0' '0' '1' '1' '0' '1' '0' '0' '0' '0' '1' '1' '0' '0' '0' '1' '0'
 '0' '1' '0' '0' '0' '1' '1' '1' '1' '1' '0' '0' '1' '0' '1' '1' '0' '1'
 '0' '0' '1' '0' '1' '1' '0' '1' '0' '0' '0' '0' '0' '1' '0' '0' '0' '0'
 '0' '0' '0' '1' '1' '1' '1' '0' '0' '0' '1' '0' '1' '0' '1' '0' '0' '0'
 '1' '1' '1' '0' '0' '0' '1' '0' '1' '0' '0' '0' '0' '1' '0' '0' '1' '0'
 '0' '0' '0' '1' '1' '0' '1' '1' '0' '0' '1' '0' '0' '1' '1' '1' '1' '0'
 '0' '1' '0' '0' '1' '0' '0' '1' '0' '0' '0' '0' '1' '0' '1' '0' '0' '1'
 '0' '0' '0' '0' '0' '1' '0' '0' '1' '0' '0' '1' '0' '1' '0' '1' '0' '0'
 '0' '0' '0' '0' '0' '1']
(

In [133]:
#Evaluating on validation datasets

pred_logreg = logreg.predict(x_test)
confusion = confusion_matrix(y_test, pred_logreg)
print('Confusion matrix:\n{}'.format(confusion))


print('\n Accuracy of the classifier is',metrics.accuracy_score(y_test, pred_logreg))
print('\n Precision is',metrics.precision_score(y_test, pred_logreg, pos_label='1'))
print('\n Recall score is',metrics.recall_score(y_test, pred_logreg, pos_label='1'))
print('\n F1 score is',metrics.f1_score(y_test, pred_logreg, pos_label='1'))

Confusion matrix:
[[103  17]
 [ 26  94]]

 Accuracy of the classifier is 0.8208333333333333

 Precision is 0.8468468468468469

 Recall score is 0.7833333333333333

 F1 score is 0.8138528138528137


In [126]:
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(2160, 2081) (240, 2081) (2160,) (240,)
